In [2]:
import json

with open("entityid2label.json", "r", encoding="utf-8") as f:
    entityid2label = json.load(f)

In [10]:
# read all tsv files
from glob import glob
import csv

paths = []

# 0.3
# Read all TSV files and collect paths
for tsv_filepath in glob("./foo/*/*.tsv"):
    with open(tsv_filepath, "r", encoding="utf-8", newline="") as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        for row in tsv_reader:
            paths.append(row)

print(f"Total paths: {len(paths)}")

# Sort paths by length
paths.sort(key=len)

# Get the shortest and longest paths
shortest_paths = paths[:100]
longest_paths = paths[-100:]

# Print shortest paths
print("\nTop 10 Shortest Paths:")
for path in shortest_paths:
    path_labels = [entityid2label.get(entityid, entityid) for entityid in path]
    print(path_labels)

# Print longest paths
print("\nTop 10 Longest Paths:")
for path in longest_paths:
    path_labels = [entityid2label.get(entityid, entityid) for entityid in path]
    print(path_labels)


Total paths: 879823

Top 10 Shortest Paths:
['editorial']
['academic journal article', 'retracted paper']
['dune', 'coastal dune']
['dune', 'singing sand']
['compound surname', 'affixed family name']
['family name', 'family name based on given name']
['family name', 'Georgian name']
['family name', 'feminine family name']
['family name', 'patronymic family name']
['family name', 'family name of Ukrainian origin']
['family name', 'masculine family name']
['family name', 'feminine family name']
['municipal service', 'municipal library']
['independent school', 'independent school']
['thesis', 'doctoral thesis']
['dissertation', 'doctoral thesis']
['family name', 'masculine family name']
['compound surname', 'double family name']
['academic journal article', 'retracted paper']
['mining region', 'oil field']
['scholarly article', 'working paper']
['scholarly article', 'medical scholarly article']
['scholarly article', 'Reproducibility Report']
['thesis', "master's thesis"]
['township of Chi

In [11]:
# read all tsv files
from glob import glob
import csv

paths = []

# 0.2
# Read all TSV files and collect paths
for tsv_filepath in glob("./bar/*/*.tsv"):
    with open(tsv_filepath, "r", encoding="utf-8", newline="") as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        for row in tsv_reader:
            paths.append(row)

print(f"Total paths: {len(paths)}")

# Sort paths by length
paths.sort(key=len)

# Get the shortest and longest paths
shortest_paths = paths[:100]
longest_paths = paths[-100:]

# Print shortest paths
print("\nTop 10 Shortest Paths:")
for path in shortest_paths:
    path_labels = [entityid2label.get(entityid, entityid) for entityid in path]
    print(path_labels)

# Print longest paths
print("\nTop 10 Longest Paths:")
for path in longest_paths:
    path_labels = [entityid2label.get(entityid, entityid) for entityid in path]
    print(path_labels)


Total paths: 4220500

Top 10 Shortest Paths:
['editorial']
['academic journal article', 'retracted paper']
['compound surname', 'affixed family name']
['family name', 'patronymic family name']
['family name', 'Georgian name']
['family name', 'family name of Ukrainian origin']
['family name', 'family name based on given name']
['family name', 'masculine family name']
['family name', 'feminine family name']
['family name', 'feminine family name']
['municipal service', 'municipal library']
['dissertation', 'doctoral thesis']
['thesis', 'doctoral thesis']
['family name', 'masculine family name']
['compound surname', 'double family name']
['academic journal article', 'retracted paper']
['mining region', 'oil field']
['scholarly article', 'Reproducibility Report']
['scholarly article', 'working paper']
['scholarly article', 'medical scholarly article']
['thesis', "master's thesis"]
['township of China', 'former township in PRC']
['conference paper', 'scholarly conference abstract']
['confere

In [ ]:
# read all tsv files
from glob import glob
import csv
paths = []

for tsv_filepath in glob("./paths/num_classes_10/MAX_PATHS_PER_CLASS_100/allowed_threshold_None/remove_the_last_downward_path_False/*/*.tsv"):
    with open(tsv_filepath, "r", encoding="utf-8", newline="") as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        for row in tsv_reader:
            paths.append(row)

print(f"total paths: {len(paths)}")
for path in paths:
    path_labels = [entityid2label.get(entityid, entityid) for entityid in path]
    print(path_labels)

In [ ]:
# read all tsv files
from glob import glob
import csv
paths = []

for tsv_filepath in glob("./paths/num_classes_10/MAX_PATHS_PER_CLASS_100/allowed_threshold_None/remove_the_last_downward_path_True/*/*.tsv"):
    with open(tsv_filepath, "r", encoding="utf-8", newline="") as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        for row in tsv_reader:
            paths.append(row)

print(f"total paths: {len(paths)}")
for path in paths:
    path_labels = [entityid2label.get(entityid, entityid) for entityid in path]
    print(path_labels)

## Test

In [ ]:

################################################################################
# 0. Define Special Tokens
################################################################################

PAD_TOKEN_ID = 0
BOS_TOKEN_ID = 1
EOS_TOKEN_ID = 2
DIR_FWD_TOKEN_ID = 3
DIR_BWD_TOKEN_ID = 4

###############################################################################
# 6. Generation Examples
###############################################################################
import json
import random
import torch
from transformers import GPT2LMHeadModel, GPT2Config


def load_some_dicts(num_classes=10):
    # Paths to JSON data (adjust as needed)
    with open(f"process_paths/vocab_{num_classes}.json", "r", encoding="utf-8") as f:
        id2label = json.load(f)

    # We shift real token IDs by +5 so that 0..4 are free for [PAD, BOS, EOS, DIR_FWD, DIR_BWD].
    # Original enumerations gave i=0.. => now i+5 => real IDs start at 5
    id2idx = {id_str: (i + 5) for i, id_str in enumerate(id2label)}
    idx2id = {v: k for k, v in id2idx.items()}  # inverse map

    return id2idx, idx2id, id2label


def generate_example(
    direction="forward",
    conditioning_token=None,
    do_sample=True,
    max_len=64,
    num_return_sequences=1,
) -> list:
    """
    Generates a sample sequence given a direction: 'forward' or 'backward'.
    direction -> which token to start with: DIR_FWD_TOKEN_ID or DIR_BWD_TOKEN_ID

    Args:
        direction: 'forward' or 'backward'
        conditioning_token: If not None, use this as the first token in the prompt.
        do_sample: Whether to sample or take the most likely token.
        max_len: Maximum length of the generated sequence.
        num_return_sequences: Number of sequences to generate.

    Returns:
        A list of lists of token IDs.
    """
    if direction.lower().startswith("f"):
        direction_token = DIR_FWD_TOKEN_ID
    else:
        direction_token = DIR_BWD_TOKEN_ID

    if conditioning_token is None:
        prompt_ids = torch.tensor(
            [[direction_token, BOS_TOKEN_ID]], dtype=torch.long, device=device
        )
    else:
        prompt_ids = torch.tensor(
            [[direction_token, BOS_TOKEN_ID, conditioning_token]],
            dtype=torch.long,
            device=device,
        )

    with torch.no_grad():

        outputs = model.generate(
            input_ids=prompt_ids,
            max_length=max_len,  # Longer for better coherence
            do_sample=do_sample,  # Stochastic sampling
            top_k=50,  # Avoid very low-probability tokens
            top_p=0.92,  # Nucleus sampling for dynamic token selection
            temperature=0.8,  # Moderate randomness
            repetition_penalty=1.2,  # Avoid repetitive sequences
            num_return_sequences=num_return_sequences,  # Adjust if you want multiple generations
            pad_token_id=PAD_TOKEN_ID,
            bos_token_id=BOS_TOKEN_ID,
            eos_token_id=EOS_TOKEN_ID,
        )

        # outputs = model.generate(
        #     input_ids=prompt_ids,
        #     max_length=max_len,
        #     do_sample=do_sample,
        #     num_return_sequences=num_return_sequences,
        #     pad_token_id=PAD_TOKEN_ID,  # So it can pad if needed
        #     bos_token_id=BOS_TOKEN_ID,
        #     eos_token_id=EOS_TOKEN_ID,
        # )
    return [item.tolist() for item in outputs]

# Define the checkpoint directory
checkpoint_dir = "model_output/checkpoint-223000"

# Load the model configuration
config = GPT2Config.from_pretrained(checkpoint_dir)

# Load the trained model
model = GPT2LMHeadModel.from_pretrained(checkpoint_dir, config=config)

# Move model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
model.to(device)
model.eval()

print(f"Model loaded from {checkpoint_dir}")

id2idx, idx2id, id2label = load_some_dicts(num_classes=100)

In [ ]:
conditioning_token = random.choice(list(idx2id.keys()))
# conditioning_token = None
max_len = 64
num_return_sequences = 1
do_sample = True

# Let's generate a few samples, forward and backward
print("=== Downstream Generation ===")
forward_ids_list = generate_example(
    direction="forward",
    conditioning_token=conditioning_token,
    max_len=max_len,
    num_return_sequences=num_return_sequences,
    do_sample=do_sample,
)
for forward_ids in forward_ids_list:
    print("Token IDs:", forward_ids)
    # Convert to your original IDs if you want to interpret them (skip 0..4)
    forward_entity_ids = [idx2id[idx] for idx in forward_ids if idx >= 5]
    forward_labels = [id2label[e] for e in forward_entity_ids]
    print("Decoded entity IDs:", forward_entity_ids)
    print("Decoded labels:", forward_labels)
    print()

print("=== Upstream Generation ===")
backward_ids_list = generate_example(
    direction="backward",
    conditioning_token=conditioning_token,
    max_len=max_len,
    num_return_sequences=num_return_sequences,
    do_sample=do_sample,
)
for backward_ids in backward_ids_list:
    print("Token IDs:", backward_ids)
    # Convert to your original IDs if you want to interpret them
    backward_entity_ids = [idx2id[idx] for idx in backward_ids if idx >= 5]
    backward_labels = [id2label[e] for e in backward_entity_ids]
    print("Decoded entity IDs:", backward_entity_ids)
    print("Decoded labels:", backward_labels)
    print()